<a href="https://colab.research.google.com/github/Faaiq123/hospital-mng.c/blob/main/5c_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Data Preprocessing

import cv2
import os
import numpy as np
import glob
from skimage import exposure
from sklearn.model_selection import train_test_split

# Function to apply CLAHE preprocessing to the MRI images
def apply_clahe(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image)

# Load images and masks, preprocess using CLAHE
def load_and_preprocess_data(img_dir, mask_dir):
    images = []
    masks = []

    for img_file in glob.glob(f'{img_dir}/*.tif'):
        image = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
        preprocessed_image = apply_clahe(image)
        images.append(preprocessed_image)

        mask_file = img_file.replace(img_dir, mask_dir)
        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
        if mask is not None:
            masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    return images, masks

# Load data
img_dir = 'path_to_images'
mask_dir = 'path_to_masks'
images, masks = load_and_preprocess_data(img_dir, mask_dir)

# Split dataset into 80% training and 20% testing sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0



In [ ]:
# Model Implementation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, concatenate, Input
from tensorflow.keras.models import Model

def unet_plus_plus(input_shape):
    inputs = Input(input_shape)

    # Encoder path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder path with nested connections
    up1 = UpSampling2D(size=(2, 2))(conv2)
    merge1 = concatenate([conv1, up1], axis=-1)
    conv3 = Conv2D(64, 3, activation='relu', padding='same')(merge1)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv3)

    model = Model(inputs, outputs)
    return model

In [ ]:
 #Model Training and Evaluation
import tensorflow as tf

# Define the DICE score metric
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

# Compile and train the models
model_unet_plus_plus = unet_plus_plus(input_shape=(256, 256, 1))
model_unet_plus_plus.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

model_attention_unet = attention_unet(input_shape=(256, 256, 1))
model_attention_unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])

# Train the models
history_unet_plus_plus = model_unet_plus_plus.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))
history_attention_unet = model_attention_unet.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Evaluate the models
dice_unet_plus_plus = model_unet_plus_plus.evaluate(X_test, y_test)
dice_attention_unet = model_attention_unet.evaluate(X_test, y_test)

print(f'Nested U-Net DICE Score: {dice_unet_plus_plus[1]}')
print(f'Attention U-Net DICE Score: {dice_attention_unet[1]}')









In [ ]:
#Web Application Development
from fastapi import FastAPI, UploadFile, File
from tensorflow.keras.models import load_model
import numpy as np
import cv2

app = FastAPI()

model = load_model('best_model.h5', custom_objects={'dice_coef': dice_coef})

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    contents = await file.read()
    nparr = np.fromstring(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)

    img = cv2.resize(img, (256, 256)) / 255.0
    img = np.expand_dims(img, axis=[0, -1])

    pred_mask = model.predict(img)
    return {"mask": pred_mask.tolist()}

In [ ]:
import streamlit as st
import requests
import numpy as np
import cv2
from PIL import Image

st.title("Brain MRI Metastasis Segmentation")

uploaded_file = st.file_uploader("Choose a Brain MRI image...", type=["jpg", "png", "tif"])

if uploaded_file is not None:
    image = np.array(Image.open(uploaded_file))
    st.image(image, caption='Uploaded MRI.', use_column_width=True)

    # Call the FastAPI backend for prediction
    files = {'file': uploaded_file.getvalue()}
    response = requests.post("http://127.0.0.1:8000/predict/", files=files)
    pred_mask = np.array(response.json()['mask'])

    st.image(pred_mask, caption='Predicted Segmentation.', use_column_width=True)



